In [ ]:
!gdown --id '1DvrMCxzJrqKKN-rU-klk7_XikfhaCr3j'

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1DvrMCxzJrqKKN-rU-klk7_XikfhaCr3j
To: /content/pdi-2022-main.zip
100% 52.6M/52.6M [00:00<00:00, 79.6MB/s]


In [ ]:
!unzip "pdi-2022-main.zip" -d "./"

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.io import read_image
import os

In [ ]:
PROCESSED_IMAGES_PATH = 'pdi-2022-main/data/processed'
ORIGINAL_IMAGES_PATH = 'pdi-2022-main/data/original'

# Criação do Dataset

In [ ]:
def get_image_path(path):
    paths = []
    if(os.path.isdir(path)):
        for child in os.listdir(path):
             paths += get_image_path(path + os.path.sep + child)
    else:
         print(path.split())
         return [path]
   
    return paths

In [ ]:
def get_masks_path(path):
    paths = []
    if(os.path.isdir(path)):
        for child in os.listdir(path):
            if child != 'images':
                paths += get_masks_path(path + os.path.sep + child)
    else:
         return [path]
   
    return paths

In [ ]:
sorted(get_image_path(PROCESSED_IMAGES_PATH))

In [ ]:
from torchvision.io import read_image

class ProcessedImagesDataset(Dataset):
    def __init__(self, processed_img_dir, original_img_dir):
        self.processed_img_dir = processed_img_dir
        self.img_paths = sorted(get_image_path(self.processed_img_dir))
        self.masks_path = sorted(get_masks_path(original_img_dir))

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = read_image(img_path)
        mask_path = self.img_masks_path[idx]
        mask = read_image(mask_path)
        
        return image, mask

In [ ]:
dataset = ProcessedImagesDataset(PROCESSED_IMAGES_PATH, ORIGINAL_IMAGES_PATH)

In [ ]:
dataset.__len__()

72

# Criação do Dataloader

In [ ]:
from torch.utils.data import DataLoader, random_split


train_data, test_data = random_split(dataset, [50, 22])

train_dataloader = DataLoader(train_data, batch_size=6, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=6, shuffle=True)

# Treino

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def check_accuracy(loader, model, device):
    num_correct = 0
    num_pixels = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device).unsqueeze(1) # Label doesn't have a channel

            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum())/((preds + y).sum() + 1e-8)

    print(f"Dice score: {dice_score/ len(loader)}")
    print(f"Got {num_correct}/{num_pixels} with accuracy {num_correct*100/num_pixels:.2f}")

In [ ]:
import torch.optim as optim

def train_fn(loader, model, optimizer, loss_fn, scaler, epochs, device):
    
    for epoch in range(epochs):
        for batch_idx, (data, targets) in enumerate(loader):
            data = data.to(device)
            targets = targets.float().unsqueeze(1).to(device)

            # forward
            with torch.cuda.amp.autocast():
                predictions = model(data)
                loss = loss_fn(predictions, targets)

            # backward
            optim.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()


        print (f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Teste